In [1]:
import torch
from resnet import ResNet50  # 上記のコードを resnet.py として保存している前提

# モデル構築
model = ResNet50()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 事前学習済み重みのロード
ckpt = torch.load("/mnt/data1/gotou/dp/resnet-50/weights.pt", map_location=device)

# "module." を除去
if any(k.startswith("module.") for k in ckpt.keys()):
    ckpt = {k.replace("module.", ""): v for k, v in ckpt.items()}

model.load_state_dict(ckpt)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): Bottleneck(
      (

In [2]:
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.ToTensor(),
])

testset = torchvision.datasets.CIFAR10(
    root="./data", train=False, download=True, transform=transform
)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False)


In [3]:
correct = 0
total = 0

with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

print(f"Accuracy: {100.0 * correct / total:.2f}%")


Accuracy: 96.04%


In [4]:
import torch
import torch.nn as nn

def fgsm_attack(model, images, labels, eps=0.03):
    images = images.clone().detach().to(device)
    labels = labels.clone().detach().to(device)
    images.requires_grad = True

    outputs = model(images)
    loss = nn.CrossEntropyLoss()(outputs, labels)
    model.zero_grad()
    loss.backward()
    grad = images.grad.data.sign()

    adv_images = images + eps * grad
    adv_images = torch.clamp(adv_images, 0, 1)  # 画素値を [0,1] に制限
    return adv_images.detach()

# Step 1: 正解した画像だけを収集
correct_images = []
correct_labels = []

model.eval()
with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        preds = outputs.argmax(1)
        mask = preds.eq(labels)  # 正解したサンプルだけ
        correct_images.append(images[mask])
        correct_labels.append(labels[mask])

correct_images = torch.cat(correct_images)
correct_labels = torch.cat(correct_labels)
print("正解サンプル数:", len(correct_labels))

# Step 2: FGSM 攻撃を適用
adv_images = []
batch_size = 128
for i in range(0, len(correct_labels), batch_size):
    imgs = correct_images[i:i+batch_size]
    lbls = correct_labels[i:i+batch_size]
    adv_batch = fgsm_attack(model, imgs, lbls, eps=0.03)
    adv_images.append(adv_batch)

adv_images = torch.cat(adv_images)

# Step 3: 精度を再評価
adv_dataset = torch.utils.data.TensorDataset(adv_images, correct_labels)
adv_loader = torch.utils.data.DataLoader(adv_dataset, batch_size=128, shuffle=False)

correct = 0
total = 0
model.eval()
with torch.no_grad():
    for images, labels in adv_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        preds = outputs.argmax(1)
        correct += preds.eq(labels).sum().item()
        total += labels.size(0)

print("Adversarial Accuracy: {:.2f}%".format(100 * correct / total))


正解サンプル数: 9604
Adversarial Accuracy: 54.64%


In [6]:
import torch
import sys
import yaml
from ml_collections import ConfigDict

sys.path.append("/mnt/data1/gotou/DiffPure")
from score_sde.models.ncsnpp import NCSNpp

device = "cuda" if torch.cuda.is_available() else "cpu"

# 設定ファイル読み込み
with open("/mnt/data1/gotou/DiffPure/configs/cifar10.yml") as f:
    cfg_dict = yaml.safe_load(f)

# dict -> ConfigDict に変換
config = ConfigDict(cfg_dict)

# モデル初期化
model = NCSNpp(config).to(device)

# チェックポイント読み込み
ckpt_path = "/mnt/data1/gotou/checkpoint_8.pth"
ckpt = torch.load(ckpt_path, map_location=device)
if 'model' in ckpt:
    model.load_state_dict(ckpt['model'])
else:
    model.load_state_dict(ckpt)
model.eval()

RuntimeError: Error(s) in loading state_dict for NCSNpp:
	Missing key(s) in state_dict: "sigmas", "all_modules.0.weight", "all_modules.0.bias", "all_modules.1.weight", "all_modules.1.bias", "all_modules.2.weight", "all_modules.2.bias", "all_modules.3.GroupNorm_0.weight", "all_modules.3.GroupNorm_0.bias", "all_modules.3.Conv_0.weight", "all_modules.3.Conv_0.bias", "all_modules.3.Dense_0.weight", "all_modules.3.Dense_0.bias", "all_modules.3.GroupNorm_1.weight", "all_modules.3.GroupNorm_1.bias", "all_modules.3.Conv_1.weight", "all_modules.3.Conv_1.bias", "all_modules.4.GroupNorm_0.weight", "all_modules.4.GroupNorm_0.bias", "all_modules.4.Conv_0.weight", "all_modules.4.Conv_0.bias", "all_modules.4.Dense_0.weight", "all_modules.4.Dense_0.bias", "all_modules.4.GroupNorm_1.weight", "all_modules.4.GroupNorm_1.bias", "all_modules.4.Conv_1.weight", "all_modules.4.Conv_1.bias", "all_modules.5.GroupNorm_0.weight", "all_modules.5.GroupNorm_0.bias", "all_modules.5.Conv_0.weight", "all_modules.5.Conv

In [ ]:
# 整理インポート
import torch
from functools import partial
from score_sde import sampling as sampling_module   # あなたが貼った sampling ファイルのモジュール
from score_sde import sde_lib
from score_sde.models import utils as mutils       # get_score_fn 等を使うため
import math

device = "cuda" if torch.cuda.is_available() else "cpu"


In [ ]:
import torch
from score_sde.models.ncsnpp import NCSNpp
from ml_collections import ConfigDict
import yaml

device = "cuda" if torch.cuda.is_available() else "cpu"

# --- 1. 拡散モデルのロード ---
with open("/mnt/data1/gotou/DiffPure/configs/cifar10.yml") as f:
    cfg_dict = yaml.safe_load(f)
config = ConfigDict(cfg_dict)

score_model = NCSNpp(config).to(device)
ckpt = torch.load("/mnt/data1/gotou/checkpoint_8.pth", map_location=device)
if "model" in ckpt:
    score_model.load_state_dict(ckpt["model"], strict=False)
else:
    score_model.load_state_dict(ckpt, strict=False)
score_model.eval()

print("拡散モデルロード完了")

拡散モデルロード完了


In [ ]:
# utils.py
from score_sde import sde_lib

def get_sde(config):
    """Return an SDE instance based on config."""
    if config.training.sde.lower() == 'vp':
        return sde_lib.VPSDE(beta_min=config.model.beta_min,
                             beta_max=config.model.beta_max,
                             N=config.model.num_scales)
    elif config.training.sde.lower() == 've':
        return sde_lib.VESDE(sigma_min=config.model.sigma_min,
                             sigma_max=config.model.sigma_max,
                             N=config.model.num_scales)
    elif config.training.sde.lower() == 'subvp':
        return sde_lib.subVPSDE(beta_min=config.model.beta_min,
                                beta_max=config.model.beta_max,
                                N=config.model.num_scales)
    else:
        raise NotImplementedError(f"Unknown SDE type: {config.training.sde}")


In [ ]:
import importlib
import utils_sde
importlib.reload(utils_sde)  # 修正版を反映
from utils_sde import get_sde


In [ ]:
from score_sde.sampling import get_pc_sampler

sde = get_sde(config)

# sampler を作るときは score_model は渡さない
sampler = get_pc_sampler(
    sde,
    shape=(1, 3, 32, 32),
    predictor="reverse_diffusion",
    corrector="langevin",
    snr=0.16,
    n_steps=1,
    probability_flow=False,
    device=device,
    inverse_scaler=lambda x: x
)

# 実際に使うときに score_model を渡す
@torch.no_grad()
def purify_batch(x_adv):
    x_adv = x_adv.to(device)
    denoised, _ = sampler(score_model, x_adv)  # ← ここで score_model を渡す
    return torch.clamp(denoised, 0, 1).cpu()
